## Part 0. Package setting
The way to do this is to manually check how to install the packages in your environment. 
this worked on collab but I installed the packages manually in my machine, the porcedure to get them will likely ghe different depending on your environment

In [ ]:
#  Install necessary libraries
!pip install transformers torch torchaudio pyaudio pydub huggingface_hub librosa soundfile -q

In [47]:
!pip install yt-dlp sentencepiece -q

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# I seems that some libraries are needed in order to install pyaudio.
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio pydub

In [9]:
# Import libraries
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer
import pyaudio
import wave
import librosa
import soundfile as sf
from pydub import AudioSegment
from IPython.display import Audio
import os
from scipy.signal import resample

In [10]:
# This part seems necessary for later

import evaluate
from jiwer import cer

def down_sample_audio(audio_original, original_sample_rate):
    target_sample_rate = 16000

    # Calculate the number of samples for the target sample rate
    num_samples = int(len(audio_original) * target_sample_rate / original_sample_rate)

    # Resample the audio array to the target sample rate
    downsampled_audio = resample(audio_original, num_samples)

    return downsampled_audio

## Part 1: Loading model and evaluating

In [11]:
# Load every part of the small model separately

MODEL_REPO = "SchindleriaPraematurus/whisper-tiny-gn-finetuned" # Your repository
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


SMALL_TOKENIZER_COMMIT_HASH = "6cd67df1b6923465434cee5bb80aadfe7a8d74ab"  # Commit for "Upload tokenizer"
SMALL_FEATURE_EXTRACTOR_COMMIT_HASH = "a7f430d0d46753348597fb241734d341c48fc69b" # Commit for "Upload feature_extractor"
SMALL_MODEL_COMMIT_HASH = "06294d5a001f1f32290e98cf9a4a6006f1fc394a" # Commit for "Upload fine-tuned Whisper-small"

In [23]:
# Movin to CPU because I ran our of ram LOL (Change this)
DEVICE='cpu' 

In [24]:


print(f"Attempting to load 'small' version components from specific commits:")
print(f" - Tokenizer commit: {SMALL_TOKENIZER_COMMIT_HASH}")
print(f" - Feature Extractor commit: {SMALL_FEATURE_EXTRACTOR_COMMIT_HASH}")
print(f" - Model commit: {SMALL_MODEL_COMMIT_HASH}")

tokenizer_small = None
feature_extractor_small = None
model_small = None
processor_small = None

try:
    # Load Tokenizer
    # These settings are usually saved in tokenizer_config.json.
    print(f"\nLoading small tokenizer from revision: {SMALL_TOKENIZER_COMMIT_HASH}...")
    tokenizer_small = WhisperTokenizer.from_pretrained(
        MODEL_REPO,
        revision=SMALL_TOKENIZER_COMMIT_HASH
    )
    print("Successfully loaded 'small' tokenizer.")

    # Load Feature Extractor
    print(f"\nLoading small feature extractor from revision: {SMALL_FEATURE_EXTRACTOR_COMMIT_HASH}...")
    feature_extractor_small = WhisperFeatureExtractor.from_pretrained(
        MODEL_REPO,
        revision=SMALL_FEATURE_EXTRACTOR_COMMIT_HASH
    )
    print("Successfully loaded 'small' feature extractor.")

    # Load Model
    print(f"\nLoading small model from revision: {SMALL_MODEL_COMMIT_HASH}...")
    model_small = WhisperForConditionalGeneration.from_pretrained(
        MODEL_REPO,
        revision=SMALL_MODEL_COMMIT_HASH
    ).to(DEVICE)
    model_small.eval()
    print("Successfully loaded 'small' model.")

    # Manually create the WhisperProcessor instance
    if tokenizer_small and feature_extractor_small:
        processor_small = WhisperProcessor(feature_extractor=feature_extractor_small, tokenizer=tokenizer_small)
        print("\nSuccessfully created 'small' WhisperProcessor manually.")
    else:
        print("\nCould not create 'small' processor because tokenizer or feature extractor failed to load.")

except Exception as e:
    print(f"\nAn error occurred during loading of 'small' components: {e}")
    print("Please double-check the following:")
    print("1. All three commit hashes are correct and point to the respective uploads for the 'small' version.")
    print(f"2. The tokenizer files (vocab.json, tokenizer_config.json, etc.) are indeed present at commit {SMALL_TOKENIZER_COMMIT_HASH}.")
    print(f"3. The feature extractor's preprocessor_config.json is present at commit {SMALL_FEATURE_EXTRACTOR_COMMIT_HASH}.")
    print(f"4. The model files (pytorch_model.bin, config.json) are present at commit {SMALL_MODEL_COMMIT_HASH}.")

if model_small and processor_small:
    print(f"\nSuccessfully prepared 'small' model (from multiple commits) for comparison.")
    # You can now use model_small and processor_small for predictions
else:
    print(f"\nFailed to fully load the 'small' model and processor. Check error messages above.")

Attempting to load 'small' version components from specific commits:
 - Tokenizer commit: 6cd67df1b6923465434cee5bb80aadfe7a8d74ab
 - Feature Extractor commit: a7f430d0d46753348597fb241734d341c48fc69b
 - Model commit: 06294d5a001f1f32290e98cf9a4a6006f1fc394a

Loading small tokenizer from revision: 6cd67df1b6923465434cee5bb80aadfe7a8d74ab...
Successfully loaded 'small' tokenizer.

Loading small feature extractor from revision: a7f430d0d46753348597fb241734d341c48fc69b...
Successfully loaded 'small' feature extractor.

Loading small model from revision: 06294d5a001f1f32290e98cf9a4a6006f1fc394a...
Successfully loaded 'small' model.

Successfully created 'small' WhisperProcessor manually.

Successfully prepared 'small' model (from multiple commits) for comparison.


## Part 2. Getting audio as input

In [36]:
# Cell 4: Record audio for a fixed duration
FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
CHANNELS = 1              # Number of audio channels (1 for mono, 2 for stereo)
RATE = 32000              # Sample rate (Whisper expects 16kHz)
CHUNK = 1024              # Number of frames per buffer
RECORD_SECONDS = 10        # Duration of recording in seconds
WAV_FILENAME = "recorded_audio.wav"
MP3_FILENAME = "recorded_audio.mp3"

audio = pyaudio.PyAudio()

In [67]:

# Start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

print("Recording...")
frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Finished recording.")

# Stop Recording
stream.stop_stream()
stream.close()
audio.terminate()

# Save the recorded data as a WAV file
wf = wave.open(WAV_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

print(f"Audio saved as {WAV_FILENAME}")

# Convert WAV to MP3 using pydub
try:
    sound = AudioSegment.from_wav(WAV_FILENAME)
    sound.export(MP3_FILENAME, format="mp3")
    print(f"Converted to {MP3_FILENAME}")
    os.remove(WAV_FILENAME) # Optional: remove the intermediate WAV file
    # Display the audio player in the notebook
    display(Audio(MP3_FILENAME))
except Exception as e:
    print(f"Could not convert to MP3. Make sure ffmpeg is installed and in your PATH. Error: {e}")
    print("You can still use the WAV file for prediction if MP3 conversion fails, but ensure your prediction block loads the correct file type.")

Recording...
Finished recording.
Audio saved as recorded_audio.wav
Converted to recorded_audio.mp3


## Part 3. Geting predictions with new model

In [14]:
from transformers import pipeline

pipe = pipeline(
    "automatic-speech-recognition",
    model=model_small,
    tokenizer=tokenizer_small,
    feature_extractor=feature_extractor_small,
    chunk_length_s=30,     # 30s chunks
    stride_length_s=10,    # 10s overlap
    device='cpu',
)


Device set to use cpu


In [ ]:
file_path = "recorded_audio.mp3"
result = pipe(file_path)
result

In [16]:
def transcribe_single_mp3(file_path, model, feature_extractor, tokenizer, gen_config, down_sample_fn):
    """
    Transcribes a single audio file using the provided Whisper components,
    mimicking the dataset processing loop.
    """
    try:
        # 1. Load audio file using librosa
        #    sr=None ensures loading the original sampling rate
        #    mono=True ensures the audio is single channel
        print(f"Loading audio file: {file_path}...")
        audio_original, original_sample_rate = librosa.load(file_path, sr=None, mono=True)
        print(f"Audio loaded. Original sample rate: {original_sample_rate}, Duration: {len(audio_original)/original_sample_rate:.2f}s")

        # 2. Downsample the audio (if necessary)
        #print("Downsampling audio to 16kHz...")
        audio_16000 = down_sample_fn(audio_original, original_sample_rate)
        #print(f"Audio downsampled. New duration: {len(audio_16000)/16000:.2f}s")


        # 3. Extract input features
        #print("Extracting input features...")
        # The feature_extractor expects the audio to be a 1D array (which mono=True ensures)
        input_feature = feature_extractor(
            raw_speech=audio_16000,
            sampling_rate=16000,
            return_tensors='pt'
        ).input_features

        # 4. Generate transcription
        # Move input_features to the same device as the model
        device = model.device
        print(f"Moving features to device: {device}")
        input_feature = input_feature.to(device)

        print("Generating transcription...")
        with torch.no_grad():
            # Use the same gen_config that works for your dataset loop
            op = model.generate(input_feature, generation_config=gen_config)

        # 5. Decode the predicted tokens
        #print("Decoding transcription...")
        text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]

        return text_pred

    except Exception as e:
        print(f"An error occurred during transcription of {file_path}: {e}")
        import traceback
        traceback.print_exc()
        return None

In [18]:
model_small.generation_config.forced_decoder_ids = None

model_small.generation_config.language = "spanish"
model_small.generation_config.task = "transcribe"


gen_config_for_small = model_small.generation_config


In [38]:
# Specify the path to your MP3 file
# ACA TOMÁ 
mp3_file = "youtube_segment.mp3" # ESTE HACE LA TRANSCRIPCION DEL AUDIO DE YUTU

my_model = model_small
my_feature_extractor = feature_extractor_small
my_tokenizer = tokenizer_small
my_gen_config = gen_config_for_small # This is the gen_config from your working dataset loop

if 'my_model' in locals() and 'my_feature_extractor' in locals() and \
   'my_tokenizer' in locals() and 'my_gen_config' in locals():

   print(f"\nAttempting to transcribe single file: {mp3_file}")
   predicted_text = transcribe_single_mp3( mp3_file, my_model, my_feature_extractor, my_tokenizer, my_gen_config, down_sample_audio)

   if predicted_text:
       print("\n--- Predicted Transcription ---")
       print(predicted_text)
   else:
       print("\nTranscription failed for the single file.")


Attempting to transcribe single file: youtube_segment.mp3
Loading audio file: youtube_segment.mp3...
Audio loaded. Original sample rate: 48000, Duration: 4.99s


`generation_config` default values have been modified to match model-specific defaults: {'forced_decoder_ids': [[1, None], [2, 50359]]}. If this is not desired, please set these values explicitly.
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppr

Moving features to device: cpu
Generating transcription...
An error occurred during transcription of youtube_segment.mp3: You have explicitly specified `forced_decoder_ids`. Please remove the `forced_decoder_ids` argument in favour of `input_ids` or `decoder_input_ids` respectively.

Transcription failed for the single file.


Traceback (most recent call last):
  File "/tmp/ipykernel_6673/3092789144.py", line 38, in transcribe_single_mp3
    op = model.generate(input_feature, generation_config=gen_config)
  File "/home/franciscobrizuela/.local/lib/python3.13/site-packages/transformers/models/whisper/generation_whisper.py", line 774, in generate
    ) = self.generate_with_fallback(
        ~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        segment_input=segment_input,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<17 lines>...
        kwargs=kwargs,
        ^^^^^^^^^^^^^^
    )
    ^
  File "/home/franciscobrizuela/.local/lib/python3.13/site-packages/transformers/models/whisper/generation_whisper.py", line 950, in generate_with_fallback
    seek_outputs = super().generate(
        segment_input,
    ...<7 lines>...
        **generate_kwargs,
    )
  File "/home/franciscobrizuela/.local/lib/python3.13/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/home/franci

In [26]:

from transformers import pipeline

# Your ASR output text in Guarani
#asr_text_guarani = "Mba'éichapa neko'ẽ?" # Example: "How are you?"
asr_text_guarani = predicted_text

# --- Models for Guarani to Spanish Translation ---

# Model 1: Helsinki-NLP Opus MT model (generally good quality, multilingual)
# This is a broader multilingual model, but we can specify the language pair.
# It might not have a dedicated Guarani-Spanish small model,
# so we'll try a general Romance language to Spanish, or a multilingual one.
# Finding a direct small Guarani->Spanish model is challenging.
# We will use a multilingual model that supports Guarani and specify Spanish as the target.

# Larger, potentially more accurate model: NLLB (No Language Left Behind)
# NLLB models are powerful but can be large. We'll try a distilled (smaller) version.
translator_gn_es_nllb = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="grn_Latn", # Guarani using Latin script
    tgt_lang="spa_Latn",  # Spanish using Latin script
    device='cpu'
)


translation_gn_es_nllb = translator_gn_es_nllb(asr_text_guarani)
print(f"NLLB (Guarani to Spanish): {translation_gn_es_nllb[0]['translation_text']}")



Device set to use cpu


NLLB (Guarani to Spanish): Dame hoy nuestro alimento de todos los tiempos y deja de hacernos mal.
Could not load a dedicated small Guarani to Spanish model (Helsinki-NLP/opus-mt-grn-es): Helsinki-NLP/opus-mt-grn-es is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
This is common for very low-resource pairs. Relying on larger multilingual models like NLLB is often necessary.


In [32]:
asr_text_guarani = "Ambyasyeteri ndaikatuihaguére rojapo porã ore rembiapo"

translator_gn_en_nllb = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="grn_Latn", # Guarani using Latin script
    tgt_lang="eng_Latn",  # Spanish using Latin script
    device='cpu'
)

Device set to use cpu


In [34]:
asr_text_guarani = "Ambyasyeteri ndaikatuihaguére rojapoporã ore rembiapo"

translation_gn_en_nllb = translator_gn_en_nllb(asr_text_guarani)
print(f"English: {translation_gn_en_nllb[0]['translation_text']}")

translation_gn_es_nllb = translator_gn_es_nllb(asr_text_guarani)
print(f"Spanish: {translation_gn_es_nllb[0]['translation_text']}")

English: I'm sorry we couldn't do our job better
Spanish: Lamento no poder mejorar nuestro trabajo


## Part 4. Comparing model sizes

In [ ]:
# Cell 7: Load the 'tiny' version components using their specific commit hashes

MODEL_REPO = "SchindleriaPraematurus/whisper-tiny-gn-finetuned" # Your repository
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


TINY_TOKENIZER_COMMIT_HASH = "5d8e8d409cb195deb958c6733939dae432db2a52"  # Commit for "Upload tokenizer"
TINY_FEATURE_EXTRACTOR_COMMIT_HASH = "612f9f357ea4677f69fcfae2ffaa4c55534a3ac2" # Commit for "Upload feature_extractor"
TINY_MODEL_COMMIT_HASH = "712fa109774ac1a772be3c29b718785afa865a9e" # Commit for "Upload fine-tuned Whisper-tiny"


In [ ]:


print(f"Attempting to load 'tiny' version components from specific commits:")
print(f" - Tokenizer commit: {TINY_TOKENIZER_COMMIT_HASH}")
print(f" - Feature Extractor commit: {TINY_FEATURE_EXTRACTOR_COMMIT_HASH}")
print(f" - Model commit: {TINY_MODEL_COMMIT_HASH}")

tokenizer_tiny = None
feature_extractor_tiny = None
model_tiny = None
processor_tiny = None

try:
    # Load Tokenizer
    # Your training script loaded the initial tokenizer with language="spanish", task="transcribe".
    # These settings are usually saved in tokenizer_config.json.
    # Try loading without explicit language/task first, as it should use the saved config.
    print(f"\nLoading tiny tokenizer from revision: {TINY_TOKENIZER_COMMIT_HASH}...")
    tokenizer_tiny = WhisperTokenizer.from_pretrained(
        MODEL_REPO,
        revision=TINY_TOKENIZER_COMMIT_HASH
    )
    print("Successfully loaded 'tiny' tokenizer.")

    # Load Feature Extractor
    print(f"\nLoading tiny feature extractor from revision: {TINY_FEATURE_EXTRACTOR_COMMIT_HASH}...")
    feature_extractor_tiny = WhisperFeatureExtractor.from_pretrained(
        MODEL_REPO,
        revision=TINY_FEATURE_EXTRACTOR_COMMIT_HASH
    )
    print("Successfully loaded 'tiny' feature extractor.")

    # Load Model
    print(f"\nLoading tiny model from revision: {TINY_MODEL_COMMIT_HASH}...")
    model_tiny = WhisperForConditionalGeneration.from_pretrained(
        MODEL_REPO,
        revision=TINY_MODEL_COMMIT_HASH
    ).to(DEVICE)
    model_tiny.eval()
    print("Successfully loaded 'tiny' model.")

    # Manually create the WhisperProcessor instance
    if tokenizer_tiny and feature_extractor_tiny:
        processor_tiny = WhisperProcessor(feature_extractor=feature_extractor_tiny, tokenizer=tokenizer_tiny)
        print("\nSuccessfully created 'tiny' WhisperProcessor manually.")
    else:
        print("\nCould not create 'tiny' processor because tokenizer or feature extractor failed to load.")

except Exception as e:
    print(f"\nAn error occurred during loading of 'tiny' components: {e}")
    print("Please double-check the following:")
    print("1. All three commit hashes are correct and point to the respective uploads for the 'tiny' version.")
    print(f"2. The tokenizer files (vocab.json, tokenizer_config.json, etc.) are indeed present at commit {TINY_TOKENIZER_COMMIT_HASH}.")
    print(f"3. The feature extractor's preprocessor_config.json is present at commit {TINY_FEATURE_EXTRACTOR_COMMIT_HASH}.")
    print(f"4. The model files (pytorch_model.bin, config.json) are present at commit {TINY_MODEL_COMMIT_HASH}.")

if model_tiny and processor_tiny:
    print(f"\nSuccessfully prepared 'tiny' model (from multiple commits) for comparison.")
    # You can now use model_tiny and processor_tiny for predictions
else:
    print(f"\nFailed to fully load the 'tiny' model and processor. Check error messages above.")

In [16]:
model_tiny.generation_config.forced_decoder_ids = None

model_tiny.generation_config.language = "spanish"
model_tiny.generation_config.task = "transcribe"


gen_config_for_tiny = model_tiny.generation_config

NameError: name 'model_tiny' is not defined

In [2]:
from datasets import load_dataset

test_data = load_dataset("mozilla-foundation/common_voice_17_0", "gn", split="test")

In [ ]:
model_tiny.eval()


for idx in range(10):

    target = test_data[idx]['sentence']
    audio_original = test_data[idx]['audio']['array']
    original_sample_rate = test_data[idx]['audio']['sampling_rate']

    audio_16000 = down_sample_audio(audio_original, original_sample_rate)

    input_feature = feature_extractor_tiny(raw_speech=audio_16000,
                                    sampling_rate=16000,
                                    return_tensors='pt').input_features

    with torch.no_grad():
        op = model_tiny.generate(input_feature.to('cuda'), generation_config=gen_config_for_tiny)


    text_pred =  tokenizer_tiny.batch_decode(op,skip_special_tokens=True )[0]

    print(f'-------{idx}------')
    print(f'true : {target} \npred : {text_pred}')
    print('\n ')

In [ ]:
model_small.eval()

for idx in range(10):

    target = test_data[idx]['sentence']
    audio_original = test_data[idx]['audio']['array']
    original_sample_rate = test_data[idx]['audio']['sampling_rate']

    audio_16000 = down_sample_audio(audio_original, original_sample_rate)

    input_feature = feature_extractor_small(raw_speech=audio_16000,
                                    sampling_rate=16000,
                                    return_tensors='pt').input_features

    with torch.no_grad():
        op = model_small.generate(input_feature.to('cpu'), generation_config=gen_config_for_small)


    text_pred =  tokenizer_small.batch_decode(op,skip_special_tokens=True )[0]

    print(f'-------{idx}------')
    print(f'true : {target} \npred : {text_pred}')
    print('\n ')

In [43]:
model_small.eval()

for idx in range(500,550):

    target = test_data[idx]['sentence']
    audio_original = test_data[idx]['audio']['array']
    original_sample_rate = test_data[idx]['audio']['sampling_rate']

    audio_16000 = down_sample_audio(audio_original, original_sample_rate)

    input_feature = feature_extractor_small(raw_speech=audio_16000,
                                    sampling_rate=16000,
                                    return_tensors='pt').input_features

    with torch.no_grad():
        op = model_small.generate(input_feature.to('cpu'), generation_config=gen_config_for_small)


    text_pred =  tokenizer_small.batch_decode(op,skip_special_tokens=True )[0]

    print(f'-------{idx}------')
    print(f'true : {target} \npred : {text_pred}')
    print('\n ')

-------500------
true : Héẽ, nde. 
pred : Héẽ, nde.

 
-------501------
true : Peteĩ ka'aru rombyatypaite ore rembipuru ha roguerekomíva guive. 
pred : Peteĩ ka'aru rombyatypaite ore rembipuru ha roguerekomíva guive.

 
-------502------
true : Rog̃uahẽvo upe óga pyahúpe. 
pred : Rog̃uahẽvo upe óga pyahúpe.

 
-------503------
true : Itakái 
pred : Itakái

 
-------504------
true : Remyaña rejúvo nde mba'yrumi. 
pred : Remyaña rejúvo nde mba'yrumi.

 
-------505------
true : Péva he'ipávo rohendu umi ryguasura'y opiã mbegue. 
pred : Péva he'ipávo rohendu umi ryguasura'y opiã mbegue.

 
-------506------
true : jopói hepyete 
pred : jopói hepyete

 
-------507------
true : Chemitã'írõ ahendúma upéva. 
pred : Chemitã'írõ ahendúma upéva.

 
-------508------
true : Añeha'ãmbaitéta. 
pred : Añeha'ãmbaitéta.

 
-------509------
true : cheñyrõmína che sy 
pred : cheñyrõmína che sy

 
-------510------
true : Ha pya'evoi hasykatu ñepyrũ. 
pred : APya'evoi hasykatũ ñepyrũ.

 
-------511------
true

## ADded the original model to get the output from it (untrained)

In [ ]:
from transformers import WhisperTokenizer, WhisperFeatureExtractor, WhisperForConditionalGeneration
import torch

# Define device (GPU if available, else CPU)
device = 'cpu'
print(f"Using device: {device}")

# Load tokenizer and feature extractor
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="spanish", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small", language="spanish", task="transcribe")

# Load model and move to appropriate device
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

In [ ]:
model.generation_config.forced_decoder_ids = None

model.generation_config.language = "spanish"
model.generation_config.task = "transcribe"


gen_config = model.generation_config

In [ ]:
from datasets import load_dataset

model.eval()

test_data = load_dataset("mozilla-foundation/common_voice_17_0", "gn", split="test")

for idx in range(10):

    target = test_data[idx]['sentence']
    audio_original = test_data[idx]['audio']['array']
    original_sample_rate = test_data[idx]['audio']['sampling_rate']

    audio_16000 = down_sample_audio(audio_original, original_sample_rate)

    input_feature = feature_extractor(raw_speech=audio_16000,
                                    sampling_rate=16000,
                                    return_tensors='pt').input_features

    with torch.no_grad():
        op = model.generate(input_feature.to('cpu'), generation_config=gen_config)


    text_pred =  tokenizer.batch_decode(op,skip_special_tokens=True )[0]

    print(f'-------{idx}------')
    print(f'true : {target} \npred : {text_pred}')
    print('\n ')

## Download from youtube

In [36]:
import yt_dlp
import subprocess

def download_youtube_audio_segment(video_url, start_time_str, end_time_str, output_filename="audio.mp3"):
    """
    Downloads a specific segment of audio from a YouTube video as an MP3 file.

    Args:
        video_url (str): The URL of the YouTube video.
        start_time_str (str): The start time of the segment (e.g., "00:01:30" for 1 minute 30 seconds).
                              If None, downloads from the beginning.
        end_time_str (str): The end time of the segment (e.g., "00:02:00" for 2 minutes).
                            If None, downloads until the end.
        output_filename (str, optional): The desired name for the output MP3 file.
                                         Defaults to "audio.mp3".
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',  # Temporary filename
        'quiet': True,
    }

    if start_time_str and end_time_str:
        # yt-dlp uses ffmpeg for time range extraction during postprocessing
        # We can also specify download sections directly if the server supports it (less common for precise cuts)
        # A more robust way is to download the best audio and then cut it with ffmpeg.

        # Download the full audio first (or best available section if supported by extractor)
        ydl_opts_full_audio = {
            'format': 'bestaudio/best',
            'outtmpl': f'%(id)s_full.%(ext)s',
            'quiet': True,
        }
        temp_filename = ""
        video_id = ""
        try:
            with yt_dlp.YoutubeDL(ydl_opts_full_audio) as ydl:
                info = ydl.extract_info(video_url, download=True)
                video_id = info.get('id', 'temp_video')
                # Construct the downloaded filename based on ydl_opts_full_audio
                # This assumes the extension is one of the common audio ones yt-dlp might download
                # Common extensions: m4a, webm, ogg, mp3 (though conversion to mp3 is a post-processing step)
                # We'll try to find the actual downloaded extension.
                downloaded_ext = info.get('ext', 'm4a') # Default to m4a if not found
                temp_filename = f"{video_id}_full.{downloaded_ext}"


            # Use ffmpeg to cut the segment and convert to mp3
            ffmpeg_command = [
                'ffmpeg',
                '-hide_banner', # Suppress version and build info
                '-loglevel', 'error', # Suppress all logs except errors
                '-y', #overwrite file if it exists
                '-i', temp_filename,
                '-ss', start_time_str,
                '-to', end_time_str,
                '-vn',  # No video
                '-acodec', 'libmp3lame', # Audio codec MP3
                '-q:a', '2',  # Audio quality (0-9, lower is better for VBR)
                output_filename
            ]
            subprocess.run(ffmpeg_command, check=True)
            print(f"Audio segment saved as {output_filename}")
        except yt_dlp.utils.DownloadError as e:
            print(f"Error during download: {e}")
        except subprocess.CalledProcessError as e:
            print(f"Error during ffmpeg processing: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
        finally:
            # Clean up the temporary full audio file
            if temp_filename and os.path.exists(temp_filename):
                os.remove(temp_filename)

    else: # Download the whole audio as mp3
        ydl_opts.update({
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192', # Standard MP3 quality
            }],
            'outtmpl': output_filename.replace('.mp3', ''), # yt-dlp adds the .mp3
        })
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([video_url])
            print(f"Full audio saved as {output_filename}")
        except Exception as e:
            print(f"An error occurred: {e}")


In [37]:

import os # For cleanup in the example

video_url = "https://www.youtube.com/watch?v=_znPZ7BeEFs&ab_channel=MasayoshiTakanaka-Topic" # Replace with a valid YouTube URL

# Example 1: Download a specific segment
start_segment = "00:00:00"  # 10 seconds
end_segment = "00:00:05"    # 30 seconds
output_segment_filename = "youtube_segment.mp3"
print(f"\nDownloading segment from {start_segment} to {end_segment}...")
download_youtube_audio_segment(video_url, start_segment, end_segment, output_segment_filename)

# Example 2: Download the whole audio (default behavior if start/end are None)
# output_full_filename = "full_audio.mp3"
# print(f"\nDownloading full audio...")
# download_youtube_audio_segment(video_url, None, None, output_full_filename)

# Example 3: Specify only start time (will download till the end from start_time)
# Note: The current function structure requires both start and end for segmentation,
# or neither for full download. To support only start_time and download till end,
# you'd need to get video duration first and use that as end_time_str.
# For simplicity, this example is omitted but can be added by fetching video duration using yt_dlp.

# Example 4: Specify output filename for full download
# custom_full_output = "my_favorite_song_full.mp3"
# print(f"\nDownloading full audio to a custom filename...")
# download_youtube_audio_segment(video_url, None, None, custom_full_output)


Audio segment saved as youtube_segment.mp3               


In [3]:
print(test_data.features)

{'client_id': Value(dtype='string', id=None), 'path': Value(dtype='string', id=None), 'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None), 'sentence': Value(dtype='string', id=None), 'up_votes': Value(dtype='int64', id=None), 'down_votes': Value(dtype='int64', id=None), 'age': Value(dtype='string', id=None), 'gender': Value(dtype='string', id=None), 'accent': Value(dtype='string', id=None), 'locale': Value(dtype='string', id=None), 'segment': Value(dtype='string', id=None), 'variant': Value(dtype='string', id=None)}


In [4]:
print(test_data)

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 855
})


## Mix the data from two parts to test the performance with cool graphs

In [5]:
from datasets import load_dataset, concatenate_datasets

# 1. Load your splits
dataset_name = "mozilla-foundation/common_voice_17_0"
language_code = "gn"
split1_name = "test" 
split2_name = "other" # It has some audio that doesnt have as good quality

split1_data = load_dataset(dataset_name, language_code, split=split1_name)
split2_data = load_dataset(dataset_name, language_code, split=split2_name)

# 2. Define the percentage you want from each split
percentage_from_split1 = 0.6  #6% from test split
percentage_from_split2 = 0.05  

# 3. Calculate the number of samples for each percentage
num_samples_split1 = int(len(split1_data) * percentage_from_split1)
num_samples_split2 = int(len(split2_data) * percentage_from_split2)

# 4. Select a random subset from each split
# Shuffle first to ensure randomness, then select
random_subset_split1 = split1_data.shuffle(seed=42).select(range(num_samples_split1))
random_subset_split2 = split2_data.shuffle(seed=42).select(range(num_samples_split2))

# 5. Concatenate the random subsets
final_mixed_dataset = concatenate_datasets([random_subset_split1, random_subset_split2])

# 6. (Optional) Shuffle the final combined dataset
final_mixed_dataset_shuffled = final_mixed_dataset.shuffle(seed=42)

print(f"Selected {len(random_subset_split1)} samples from {split1_name}")
print(f"Selected {len(random_subset_split2)} samples from {split2_name}")
print(f"Total samples in final mixed dataset: {len(final_mixed_dataset_shuffled)}")
print(final_mixed_dataset_shuffled)

Selected 513 samples from test
Selected 938 samples from other
Total samples in final mixed dataset: 1451
Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 1451
})


In [ ]:
def calculate_average_noise_db(audio_data):
    """
    Calculates the average signal level in dBFS for an audio clip.
    Assumes audio_data is a NumPy array from the 'audio' field.
    """
    samples = audio_data['array'] # Extract the numpy array
    if len(samples) == 0:
        return -np.inf # Or some other indicator for silence or error
    
    # Calculate RMS energy
    rms_energy = np.sqrt(np.mean(samples**2))
    
    # Convert RMS to dBFS (assuming samples are normalized between -1 and 1)
    # If RMS is 0 (silence), log10 will be undefined, handle this.
    if rms_energy == 0:
        return -np.inf # Represents negative infinity dB, common for silence
    
    dbfs = 20 * np.log10(rms_energy)
    return dbfs

# Apply the function to your dataset
# This will add a new column 'average_noise_db' to each example
updated_data = test_data.map(lambda example: {'average_noise_db': calculate_average_noise_db(example['audio'])})

# Now updated_data has the new column
print(updated_data.features)
# You should see 'average_noise_db': Value(dtype='float64', id=None) or similar

# To separate files into "buckets":
# You can then filter or sort 'updated_data' based on the 'average_noise_db' column.
# For example, to get all files with noise between -40 dB and -30 dB:
# bucket_1 = updated_data.filter(lambda example: -40 < example['average_noise_db'] <= -30)